# Lab: Understanding ReAct (🤖 Reason + ⚡ Act) with OpenAI

Welcome! In this lab, we'll explore the **ReAct** prompting technique. ReAct is a powerful pattern that enables Large Language Models (LLMs) to solve complex problems by combining reasoning and action steps.

### What is ReAct?

Instead of just asking an LLM for a final answer, we ask it to "think out loud." It breaks down a problem into a sequence of steps:

1.  **Thought 🤔:** The model reasons about the problem and decides what to do next.
2.  **Action ⚡:** The model chooses a tool or action to perform to gather information.
3.  **Observation 👀:** The result of the action is given back to the model.

This loop repeats until the model has enough information to give a final answer. It makes the model's process more transparent and often more accurate, especially for questions that require up-to-date information or calculations.

---

## 1. Setup

First, let's install the necessary library and set up our OpenAI API key.

In [ ]:
%pip install openai

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [3]:
import os
from openai import OpenAI
from getpass import getpass # To securely ask for the API key

# It's best practice to use environment variables for API keys.
# If you don't have it set, this will prompt you to enter it securely.
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

client = OpenAI()

print("OpenAI client initialized successfully!")

OpenAI client initialized successfully!


---

## 2. Standard Prompt vs. ReAct Prompt

Let's start with a simple question and see the difference between a standard prompt and a ReAct-style prompt.

**Question:** *"Who won the men's singles title at Wimbledon in 2023 and who did he beat in the final?"*

### Standard Prompt

Here, we just ask the question directly.

In [4]:
standard_prompt = "Who won the men's singles title at Wimbledon in 2023 and who did he beat in the final?"

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": standard_prompt}],
    temperature=0
)

print(response.choices[0].message.content)

Carlos Alcaraz won the men's singles title at Wimbledon in 2023. He defeated Novak Djokovic in the final.


### ReAct-style Prompt

Now, let's instruct the model to use the ReAct format. We are not using external tools yet, just asking it to show its reasoning process.

In [5]:
react_prompt = """
Answer the following question by reasoning step-by-step. Use the following format:

Thought: The user wants to know the winner and runner-up of the 2023 Wimbledon men's singles final. I need to access my internal knowledge about this event.
Final Answer: The final answer to the user's question.

Question: Who won the men's singles title at Wimbledon in 2023 and who did he beat in the final?
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": react_prompt}],
    temperature=0
)

print(response.choices[0].message.content)

Thought: The user wants to know the winner and runner-up of the 2023 Wimbledon men's singles final. I need to access my internal knowledge about this event.

Final Answer: The winner of the men's singles title at Wimbledon in 2023 was Carlos Alcaraz, and he beat Novak Djokovic in the final.


**Observation:** Notice how the ReAct prompt forces the model to lay out its thinking process. This isn't super useful yet because the model is just using its internal knowledge. The real power comes when we give it **tools** to use.

---

## 3. ReAct with Tools

Now for the fun part! We'll create a simulated environment where the LLM can use external tools. We will define two simple Python functions:

1.  `search(query)`: Simulates a web search.
2.  `calculate(expression)`: Simulates a calculator.

In [6]:
# Tool 1: A simulated search engine
def search(query: str) -> str:
    """Simulates searching the web for a given query."""
    print(f"\n🤖 SEARCHING for: '{query}'...")
    # We'll hardcode the results for this lab
    query = query.lower()
    if "when was the first iphone released" in query:
        return "The first iPhone was released on June 29, 2007."
    elif "us president in 2007" in query:
        return "George W. Bush was the US president in 2007. He was born on July 6, 1946."
    else:
        return "Sorry, I couldn't find any information on that."

# Tool 2: A simulated calculator
def calculate(expression: str) -> str:
    """Simulates a calculator that can evaluate a simple math expression."""
    print(f"\n🧮 CALCULATING: '{expression}'...")
    try:
        # A safe way to evaluate a string expression
        result = eval(expression, {"__builtins__": {}}, {})
        return f"The result is {result}."
    except Exception as e:
        return f"Error: {e}"

# Test our tools
print(search("US President in 2007"))
print(calculate("2007 - 1946"))


🤖 SEARCHING for: 'US President in 2007'...
George W. Bush was the US president in 2007. He was born on July 6, 1946.

🧮 CALCULATING: '2007 - 1946'...
The result is 61.


### The Manual ReAct Loop

Now, we'll solve a multi-step problem. We will act as the "computer" that executes the actions proposed by the LLM. We will feed the observations back to the model until it finds the final answer.

**Our complex question:** *"Who was the president of the United States when the first iPhone was released, and what was his age at that time?"*

In [7]:
# This is the core prompt that tells the LLM how to behave
react_system_prompt = """
You are a helpful assistant that answers questions by breaking them down into a series of thoughts and actions. Your goal is to find the final answer.

You have access to the following tools:
1. search(query: str): Use this to find information about current events, facts, and people.
2. calculate(expression: str): Use this to perform mathematical calculations.

Follow this format strictly:
Thought: Your reasoning about what to do next to get closer to the answer.
Action: The tool you want to use, in the format `search("your query")` or `calculate("your expression")`.

After an action, you will receive an Observation. You will then repeat the Thought/Action cycle. Once you have enough information, provide the final answer in this format:
Final Answer: [Your conclusive answer here]
"""

In [8]:
# We will store the conversation history here
messages = [
    {"role": "system", "content": react_system_prompt},
    {"role": "user", "content": "Who was the president of the United States when the first iPhone was released, and what was his age at that time?"}
]

def run_react_loop(max_steps=5):
    for i in range(max_steps):
        print(f"\n--- STEP {i+1} ---")

        # 1. Get the LLM's next Thought and Action
        print("🤔 Thinking...")
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0,
            stop=["\nObservation:"] # Stop generation when it's time for us to provide an observation
        )
        llm_output = response.choices[0].message.content
        print(llm_output)

        # Add the LLM's turn to the conversation history
        messages.append({"role": "assistant", "content": llm_output})

        # 2. Check if the LLM provided a final answer
        if "Final Answer:" in llm_output:
            print("\n✅ ReAct loop finished!")
            break

        # 3. Execute the Action
        action_str = llm_output.split("Action:")[-1].strip()
        observation = ""
        if action_str.startswith("search("):
            query = action_str[len("search("):-1].strip('"')
            observation = search(query)
        elif action_str.startswith("calculate("):
            expression = action_str[len("calculate("):-1].strip('"')
            observation = calculate(expression)
        else:
            observation = "Error: Invalid action specified."

        # 4. Provide the Observation back to the LLM
        print(f"👀 OBSERVATION: {observation}")
        messages.append({"role": "user", "content": f"Observation: {observation}"})

    else:
        print("\n⚠️ ReAct loop reached max steps.")

# Let's run it!
run_react_loop()


--- STEP 1 ---
🤔 Thinking...
Thought: To find out who was the president of the United States when the first iPhone was released, I need to know the release date of the first iPhone. Then, I can determine who was the president at that time and calculate his age.

Action: search("first iPhone release date")

🤖 SEARCHING for: 'first iPhone release date'...
👀 OBSERVATION: Sorry, I couldn't find any information on that.

--- STEP 2 ---
🤔 Thinking...
Thought: I know from prior knowledge that the first iPhone was released on June 29, 2007. Now, I need to find out who was the president of the United States on that date.

Action: search("US president June 29, 2007")

🤖 SEARCHING for: 'US president June 29, 2007'...
👀 OBSERVATION: Sorry, I couldn't find any information on that.

--- STEP 3 ---
🤔 Thinking...
Thought: I know from prior knowledge that George W. Bush was the President of the United States in 2007. Now, I need to find out his birthdate to calculate his age on June 29, 2007.

Action:

---

## 4. Conclusion & Key Takeaways

Congratulations on completing the ReAct lab! 🥳

You've successfully guided an LLM to solve a complex problem by giving it tools and a reasoning framework. 

### Key Takeaways:

1.  **Transparency:** The `Thought` process shows you *how* the model is working, making it easier to debug and trust.
2.  **Accuracy:** By using external tools (like search or a calculator), the model can access up-to-date information and perform precise calculations, overcoming its inherent limitations.
3.  **Extensibility:** You can create any tool you want! Imagine giving the model tools to check your calendar, send an email, or query a database.

**In the real world**, frameworks like **LangChain** and **LlamaIndex** automate this ReAct loop, making it much easier to build powerful, tool-augmented AI applications. However, understanding the fundamental `Thought -> Action -> Observation` cycle is the key to mastering them.